In [1]:
# Model Arguments 
from config import ModelArguments

config = ModelArguments()

# Multimodal Encoder, Resampler, Projector
from multimodal_encoder.builder import build_vision_tower
from multimodal_resampler.builder import build_vision_resampler
from multimodal_projector.builder import build_vision_projector

vision_tower = build_vision_tower(config)
vision_resampler = build_vision_resampler(config)
vision_projector = build_vision_projector(config)

On MacOS, decord package is not compatible with new python version.


/opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
from llava_llama import LlavaLlamaForCausalLM # Register the llava models into 'transformers'
from transformers import AutoConfig
from transformers import AutoModelForCausalLM

config = AutoConfig.for_model("llava_llama", trust_remote_code=True)
model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)

: 

In [2]:
# Dataset Construction for MultiModality

from dataprocess import LazySupervisedDataset
from transformers import AutoTokenizer
from config import DataArguments 

data_args = DataArguments(
    data_path = "data/mock.json",
    image_folder = "data/",
    video_folder = "data/",
    video_fps = 1,
    frames_upbound = 0,
    add_time_instruction = False,
    force_sample = False,
    default_fps = 10
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
dataset = LazySupervisedDataset(data_args=data_args, tokenizer=tokenizer, image_processor=vision_tower.image_processor)
data_dict = dataset[0]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Right, now we try to propagate the data_dict into the model 
import torch 
from constants import IGNORE_INDEX

# # Assuming you have already loaded the model
# model = LlavaLlamaForCausalLM.from_pretrained("path_to_your_model")
# model.eval()  # Set the model to evaluation mode

# Prepare the inputs
input_ids = data_dict['input_ids'].unsqueeze(0)  # Add batch dimension
labels = data_dict['labels'].unsqueeze(0)  # Add batch dimension

# Prepare images, a list of per-patch-images tensors
images = []
if 'image' in data_dict:
    batch_images = [item[0] for item in data_dict["image"]]
    batch_images = torch.cat(batch_images, dim=0)
    images = [batch_images]

# Create attention mask
attention_mask = (labels != IGNORE_INDEX).long()

# Forward pass
# outputs = model(
#     input_ids=input_ids,
#     attention_mask=attention_mask,
#     labels=labels,
#     images=images,
# )

NameError: name 'model' is not defined

In [ ]:
# Error Tracing 
outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, images=images) # Error occurs
model.prepare_inputs_labels_for_multimodal(input_ids, None, attention_mask, None, labels, images) # Same Error Propagated here

